In [6]:

import pandas as pd
from lib.constants import *
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import math
from math import pi
import ast
import seaborn as sns
from wordcloud import WordCloud
from statistics import mode
import random
import ternary

os.makedirs(OUTPUT_GRAPHS, exist_ok=True)
FONT_TITLE = 18
FONT_TICKS = 14
COLOR = 'color'
LINESTYLE = 'linestyle'
LABEL = 'label'
IBM_COLORBLINDPALETTE = ['#ffb000', '#fe6100', '#dc267f', '#785ef0', '#648fff']
MARKERS = ['o', "s", "^", "D", "X"]

MODELS_GRAPHICS = {
    BERT_BASE : {
        LABEL: 'BERT Base',
        COLOR: '#ffb000', 
        LINESTYLE: '-'
    },
    BERT_LARGE : {
        LABEL: 'BERT Large',
        COLOR: '#ffb000', 
        LINESTYLE: '--'
    },
    ROBERTA_BASE : {
        LABEL: 'RoBERTa Base',
        COLOR: '#41a326', 
        LINESTYLE: '-'
    },
    ROBERTA_LARGE : {
        LABEL: 'RoBERTa Large',
        COLOR: '#41a326', 
        LINESTYLE: '--'
    },
    LLAMA3 : {
        LABEL: 'Llama 3',
        COLOR: '#785ef0', 
        LINESTYLE: '-'
    },
    LLAMA3_70B : {
        LABEL: 'Llama 3 (70b)',
        COLOR: '#785ef0', 
        LINESTYLE: '--'
    },
    GEMMA3 : {
        LABEL: 'Gemma 3',
        COLOR: '#dc267f', 
        LINESTYLE: '-'
    },
    GEMMA3_27B : {
        LABEL: 'Gemma 3 (27b)',
        COLOR: '#dc267f', 
        LINESTYLE: '--'
    },
    GEMINI_2_0_FLASH_LITE : {
        LABEL: "Gemini 2.0 Flash Lite",
        COLOR: '#fe6100', 
        LINESTYLE: '-'
    },
    GEMINI_2_0_FLASH : {
        LABEL: "Gemini 2.0 Flash",
        COLOR: '#fe6100', 
        LINESTYLE: '--'
    },
    GPT4_MINI : {
        LABEL: 'GPT4o Mini',
        COLOR: '#648fff', 
        LINESTYLE: '-'
    },
    GPT4 : {
        LABEL: 'GPT4o',
        COLOR: '#648fff', 
        LINESTYLE: '--'
    },
    DEEPSEEK: {
        LABEL: 'DeepSeek R1',
        COLOR: '#000000', 
        LINESTYLE: '-'
    },
    DEEPSEEK_673B: {
        LABEL: 'DeepSeek R1 (671b)',
        COLOR: '#000000', 
        LINESTYLE: '--'
    }
}

In [7]:

def plot_stacked_bars_per_model(models,):
    """
    data: dict of dicts
        Example format:
        {
            'model1': {
                'Unmarked': [val1_stack1, val1_stack2, val1_stack3],
                'Non Queer': [val2_stack1, val2_stack2, val2_stack3],
                'Queer': [val3_stack1, val3_stack2, val3_stack3]
            },
            'model2': {...},
            ...
        }
    subject_categories: list of str
        Example: ['Unmarked', 'Non Queer', 'Queer']
    stack_labels: list of str
        Example: ['Regard 1', 'Regard 2', 'Regard 3']
    title: str
        Title of the overall figure
    """
            
    study = pd.DataFrame()
    y_points_list =[]
    for model in models:
        data = pd.read_csv(f"{OUTPUT_EVALUATION+model}.csv") 
        y_points =[]
        for subjCat in SUBJ_CATEGORIES:
            df = data[data[TYPE] == subjCat].copy()
            for cat in REGARD_CATEGORIES:
                scoreList = df["Regard "+cat].dropna().tolist() #Get the scores for that subject category
                scoreList = np.mean(scoreList) #Calculate the mean for that subject category scores
                y_points.append(scoreList)
        y_points_list.append(y_points)
        study[model] = {cat: point for cat, point in zip(SUBJ_CATEGORIES, y_points)}
    display(study)
    
    title='Stacked Bar Charts per Model'
    n_models = len(data)
    fig, axes = plt.subplots(1, n_models, figsize=(5 * n_models, 6), sharey=True)

    if n_models == 1:
        axes = [axes]  # Make axes iterable if there's only one model

    colors = ['#4daf4a', '#377eb8', '#ff7f00']  # Choose 3 colors for the 3 stacks

    for ax, (model_name, category_data) in zip(axes, data.items()):
        bar_positions = np.arange(len(SUBJ_CATEGORIES))
        bottoms = np.zeros(len(SUBJ_CATEGORIES))  # Track the bottom of the stacks

        for stack_idx in range(len(REGARD_CATEGORIES)):
            # Heights of the current stack for each subject
            heights = [category_data[subj][stack_idx] for subj in SUBJ_CATEGORIES]
            ax.bar(bar_positions, heights, bottom=bottoms, color=colors[stack_idx], label=REGARD_CATEGORIES[stack_idx])
            bottoms += heights

        ax.set_title(model_name, fontsize=14)
        ax.set_xticks(bar_positions)
        ax.set_xticklabels(SUBJ_CATEGORIES, rotation=45)
        ax.set_xlabel('Subject Category')
        ax.grid(axis='y', linestyle='--', alpha=0.7)

    axes[0].set_ylabel('Score')

    # Create a single legend
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', ncol=len(REGARD_CATEGORIES))

    plt.suptitle(title, fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 0.92])
    plt.show()

plot_stacked_bars_per_model([BERT_BASE, BERT_LARGE])

,BERT_base,BERT_large
Unmarked,0.1974,0.2867
Non Queer,0.7576,0.6414
Queer,0.0293,0.0477


KeyError: 'Unmarked'

Error in callback <function _draw_all_if_interactive at 0x31298e550> (for post_execute), with arguments args (),kwargs {}:


ValueError: Image size of 1550000x600 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 1550000x600 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1.55e+06x600 with 3100 Axes>